# Transformers Tutorial: Case of next location prediction in mobility sequences

Transformers are current state-of-the-art for sequence-to-sequence conversion tasks and generative AI. This lesson will illustrate the idea of transformers demonstrated over a case of next location prediction in mobility sequences. In this short Jupyter tutorial you will learn how to design, implement, train, and evaluate a transformer model in PyTorch.

We shall
* generate synthetic data - sequences of locations visited
* process the data (train-test split, creating loaders for model training, splitting in batches)
* desing, implement, train and evaluate (out-of-sample (OS) accuracy) the baseline recurrent neural network (RNN) model for the next location preduction
* desing, implement, train and evaluate (OS accuracy) the transformer model the next location preduction


## Generate the synthetic data

We generate a series of synthetic sequences of locations visited by a random walker with memory. Relative probabilities of moving from one location to another exponentially decay with distance between locations, and are further depreciated for the locations recently visited, stimulating the random walker to explore new locations, rather than getting stuck into single locations of their immediate neighborhoods.

The specific parameters of the process below won't be used by the transformer model - instead it will be up to it to figure out all the relevant features of the process in order to be able to predict the next location

In [1]:
import numpy as np

In [2]:
# Constants
N = 1000  # number of sequences
n = 100  # length of each sequence
m = 30  # number of locations
alpha = 15.0  # decay factor for distance
recency_decay_depth = 10 #depth of random walkers previous location memory window


In [3]:
# Generate random 2D coordinates for the locations
np.random.seed(0) #fix the random seed for reproducibility
XY = np.random.normal(size = (m, 2)) #standard normal distribution for XY coordinates of the locations
all_distances = np.linalg.norm(XY[:, np.newaxis, :] - XY[np.newaxis, :, :], axis=2) #define pairwide euclidian distances between locations

In [4]:
def generate_sequences(N, n, m, all_distances, recency_decay_depth, alpha): #function to generate synthetic mobility sequences
    # N - number of sequences
    # n - number of locations visited per sequence
    # m - total number of locations
    # all_distance - pairwise distances between locations
    # alpha - parameter for probability exponential decay with distance
    sequences = np.zeros((N, n), dtype=int)
    visit_times = np.full((N, m), -np.inf)  # Initialize previous visit times per location with large negative values
    current_locations = np.random.randint(m, size = N) #pick random initial locations

    # Set initial locations and times
    sequences[:, 0] = current_locations
    visit_times[np.arange(N), current_locations] = 0


    for t in range(1, n): #for each new step of the process
        # Retrieve distances for current locations
        current_distances = all_distances[current_locations, :]  #distances to possible next locations

        # Calculate probability decay factors for recently visited locations
        recency = np.maximum(recency_decay_depth + 2 + visit_times - t, 1)

        # Calculate probabilities - exponential decay with distance plus additional decay for recently visited ones (current location can't be visited again, locations visited over the previous recency_decay_depth steps get their probabilities scaled down)
        probabilities = (recency <= recency_decay_depth) * np.exp(-alpha * current_distances) / recency  # Shape (N, m)
        probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize probabilities

        # Choose next locations based on probabilities
        next_locations = np.array([np.random.choice(m, p=probabilities[i]) for i in range(N)])
        sequences[:, t] = next_locations

        # Update visit times and current locations
        visit_times[np.arange(N), next_locations] = t
        current_locations = next_locations

    return sequences

In [5]:
# Generate training sequences
sequences = generate_sequences(N, n, m, all_distances, recency_decay_depth, alpha)

In [6]:
sequences[0, :] #example of one of the generated sequences

array([23, 25, 17, 22, 29, 19, 13, 19, 22, 19, 22, 29, 22, 19, 22, 29, 22,
       19, 22, 29, 22, 19, 22, 19, 22, 29, 22, 19, 17, 28, 27, 15, 28,  4,
       27, 28, 27, 15, 28, 27,  4, 27,  4, 27,  4, 27,  4, 15, 28, 15,  7,
        6,  3,  6,  7, 28, 27,  4, 27, 15, 28, 27, 28, 27,  4, 15, 28, 27,
        4, 27, 28, 27,  4, 27,  4, 17, 29, 22, 19, 13,  9, 11,  6,  3,  6,
        7, 15, 28, 15, 28,  4, 27,  4, 27, 17, 19, 22, 29, 22, 19])

In [7]:
#import pytorch-related libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [8]:
def prepare_data(sequences, split_ratio = [0.6, 0.8]): #prepare data to be fed to the model for training and validation; perform train - test split, split into batches
    # Determine split index
    num_sequences = sequences.shape[0]
    split_index = [int(num_sequences * s) for s in split_ratio]

    # Split the sequences into training-validation/test
    train_sequences = sequences[:split_index[0]]
    val_sequences = sequences[split_index[0]:split_index[1]]
    test_sequences = sequences[split_index[1]:]


    # Convert train-validation sequences to PyTorch datasets and loaders
    # include only the
    train_dataset = TensorDataset(torch.tensor(train_sequences[:, : -1]), torch.tensor(train_sequences[:, 1:]))
    val_dataset = TensorDataset(torch.tensor(val_sequences[:, : -1]), torch.tensor(val_sequences[:, 1:]))
    test_dataset = TensorDataset(torch.tensor(test_sequences[:, : -1]), torch.tensor(test_sequences[:, 1:]))

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

# Assuming `sequences` are ready and correctly shaped
train_loader, val_loader, test_loader = prepare_data(sequences)

In [9]:
class RNNModel(nn.Module): #recurrent neural network (previous state-of-the-art baseline model to compare transformers with)
    def __init__(self, input_size, hidden_size, num_layers, output_size): #constructor, hidden_size (dimensionality of the hidden state) and num_layers (number of layers) are hyperparameters to specify
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) #defined an RNN model
        self.fc = nn.Linear(hidden_size, output_size) #define a final linear transform of an RNN's hidden state into the dimensionality of the output layer (number of locations)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x): #feedforward step
        out, _ = self.rnn(x) #convert input into hidden state over each token
        out = self.fc(out) #convert hiddent state to output dimensionality
        return out

    def train_model(self, train_loader, val_loader, epochs=10, lr=0.001): #method for model training
        criterion = nn.CrossEntropyLoss() #specify the loss function as a cross-entropy loss
        optimizer = optim.Adam(self.parameters(), lr=lr) #specify Adam optimizer

        for epoch in range(epochs): #iterate over epochs
            self.train() #first call the superclass train
            train_loss = 0
            for data, targets in train_loader: #for each batch
                data = data.float().unsqueeze(-1).to(self.device)  # Add feature dimension and send to the device
                targets = targets.long().to(self.device) #send the target labels to the device
                optimizer.zero_grad()
                output = self(data) #feedforward step
                loss = criterion(output.view(-1, m), targets.view(-1)) #define the loss function over the feedforward output
                loss.backward() #backpropagate the loss gradient
                optimizer.step() #update the weights
                train_loss += loss.item() #aggregate the values of the loss over batches


            # Assess the loss over validation data (without training)
            self.eval()
            valid_loss = 0
            with torch.no_grad():
                for data, targets in val_loader:
                    data = data.float().unsqueeze(-1).to(self.device)
                    targets = targets.long().to(self.device)
                    output = self(data)
                    loss = criterion(output.view(-1, m), targets.view(-1))
                    valid_loss += loss.item()

            print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Validation Loss: {valid_loss/len(val_loader)}')

    def evaluate(self, test_loader): #method for accuracy evaluation over a given dataset
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, targets in test_loader: #iterate over batches
                data = data.float().unsqueeze(-1).to(self.device)
                targets = targets.long().to(self.device)
                outputs = self(data)
                predicted = outputs.argmax(dim=2) #discretize output - take the most likely location token
                total += targets.size(0) * targets.size(1)
                correct += (predicted == targets).sum().item() #count the number of label matches between the predicted and actual location token

        accuracy = correct / total
        print(f'Accuracy: {accuracy}')
        return accuracy

In [10]:
# Model initialization
torch.manual_seed(0)
model = RNNModel(input_size = 1, hidden_size = 64, num_layers = 2, output_size = m)

# Assuming train_loader and val_loader have been defined
model.train_model(train_loader, test_loader, epochs = 30, lr = 0.001)

Epoch 1, Train Loss: 2.92254460485358, Validation Loss: 2.6295317241123746
Epoch 2, Train Loss: 2.48076195465891, Validation Loss: 2.3950676918029785
Epoch 3, Train Loss: 2.2627851963043213, Validation Loss: 2.2028953347887312
Epoch 4, Train Loss: 2.0916713287955835, Validation Loss: 2.0129064321517944
Epoch 5, Train Loss: 1.9276610863836188, Validation Loss: 1.8194690772465296
Epoch 6, Train Loss: 1.774414991077624, Validation Loss: 1.6585581302642822
Epoch 7, Train Loss: 1.662080256562484, Validation Loss: 1.5642338820866175
Epoch 8, Train Loss: 1.5855204557117664, Validation Loss: 1.4963739940098353
Epoch 9, Train Loss: 1.5312239810040122, Validation Loss: 1.444969722202846
Epoch 10, Train Loss: 1.482482715656883, Validation Loss: 1.4023301431110926
Epoch 11, Train Loss: 1.4404163109628778, Validation Loss: 1.361840350287301
Epoch 12, Train Loss: 1.3999385018097728, Validation Loss: 1.3279962028775896
Epoch 13, Train Loss: 1.36403825408534, Validation Loss: 1.2958445719310216
Epoch 

In [11]:
# Assuming test_loader is defined for the new sequence or a separate test set
model.evaluate(test_loader)

Accuracy: 0.6044949494949495


0.6044949494949495

So we reach the OS accuracy of over 60% with relatively little effort designing and training the model! Not so bad for predicting a random walk... Let's see if a transformer model can do it any better?

In [12]:
class TransformerModel(nn.Module): #transfomrer model
    def __init__(self, input_size, hidden_size, num_layers, output_size, nhead, max_len=1000): #initiatize with hyperparameters - dimensionality of the hidden layer (embedding), number of attention heads, number of layers, output_size - number of locations
        super(TransformerModel, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size) #introduce the trainable token embedding for all the locations
        self.output_size = output_size #number of locations
        self.positional_encoding = self._generate_positional_encoding(hidden_size, max_len) #add positional embedding
        self.transformer = nn.TransformerEncoder( #introduce the transformer model
            nn.TransformerEncoderLayer(d_model = hidden_size, nhead = nhead, dim_feedforward = hidden_size * 4),
            num_layers=num_layers)
        self.out = nn.Linear(hidden_size, output_size) #add a linear layer to convert transformer output into next token probability distribution (subject to further softmax nodmalization)
        self.to(self.device)

    def forward(self, x): #feedworward step
      # x: [batch_size, seq_length]
      x = self.embedding(x)  # Transform input to embeddings: [batch_size, seq_length, hidden_size]

      # Ensure positional_encoding is broadcastable over the batch dimension
      # x.size(1) gives the sequence length, positional_encoding needs to be sliced accordingly
      pos_encoding = self.positional_encoding[:x.size(1), :].unsqueeze(0)  # [1, seq_length, hidden_size]
      x = x + pos_encoding  #Add positional encoding to each batch of token embeddings

      x = x.permute(1, 0, 2)  # Transformer expects [seq_len, batch_size, features]
      attn_mask = torch.triu(torch.ones(x.size(0), x.size(0)) * float('-inf'), diagonal=1).to(self.device)
      x = self.transformer(x, mask = attn_mask)
      x = x.permute(1, 0, 2)  # Back to [batch_size, seq_len, features]
      return self.out(x) #apply a final linear output layer and return the output

    def _generate_positional_encoding(self, hidden_size, max_len): #trigonometric positional embedding
        pe = torch.zeros(max_len, hidden_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2).float() * (-np.log(10000.0) / hidden_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.to(self.device)

    def train_model(self, train_loader, val_loader, epochs=10, lr=0.001): #method for model training
        criterion = nn.CrossEntropyLoss() #introduce cross-entropy loss
        optimizer = optim.Adam(self.parameters(), lr=lr) #introduce adam optimizer

        for epoch in range(epochs): #iterate through epochs
            self.train()
            train_loss = 0
            for data, targets in train_loader: #iterate through batches
                data = data.long().to(self.device)
                targets = targets.long().to(self.device)
                optimizer.zero_grad()
                output = self(data) #feedforward loop
                loss = criterion(output.view(-1, self.output_size), targets.view(-1)) #loss function
                loss.backward() #backpropagate loss gradient
                optimizer.step() #update weights towards the gradient
                train_loss += loss.item() #accumulate batch loss

            # Complute validation (OS) loss
            self.eval()
            valid_loss = 0
            with torch.no_grad():
                for data, targets in val_loader:
                    data = data.long().to(self.device)
                    targets = targets.long().to(self.device)
                    output = self(data)
                    loss = criterion(output.view(-1, self.output_size), targets.view(-1))
                    valid_loss += loss.item()

            train_loss /= len(train_loader)
            valid_loss /= len(val_loader)
            
            print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {valid_loss}')
        return train_loss, valid_loss    

    def evaluate(self, test_loader): #accuracy evaluation for the given dataset
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad(): #iterate through all the batches
            for data, targets in test_loader:
                data = data.long().to(self.device)
                targets = targets.long().to(self.device)
                outputs = self(data)
                predicted = outputs.argmax(dim=2)
                total += targets.size(0) * targets.size(1)
                correct += (predicted == targets).sum().item()

        accuracy = correct / total
        print(f'Accuracy: {accuracy}')
        return accuracy

In [13]:
# Model parameters
torch.manual_seed(0)
input_size = 1  # not used in our Transformer model, kept for compatibility
hidden_size = 64  # dimension of the embedding and hidden layers
num_layers = 2  # number of transformer layers
nhead = 8  # number of heads in multi-head attention mechanisms

# Create the Transformer model instance
model2 = TransformerModel(input_size, hidden_size, num_layers, m, nhead)

/Users/stanislav/opt/anaconda3/envs/STTN/lib/python3.9/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [14]:
# Train the model
model2.train_model(train_loader, test_loader, epochs = 30, lr = 0.001)

Epoch 1, Train Loss: 2.266926332523948, Validation Loss: 1.4887442759105138
Epoch 2, Train Loss: 1.3512907279165167, Validation Loss: 1.139384354863848
Epoch 3, Train Loss: 1.1304840288664166, Validation Loss: 1.0026372160230363
Epoch 4, Train Loss: 1.0307391756459285, Validation Loss: 0.939822963305882
Epoch 5, Train Loss: 0.9808928025396246, Validation Loss: 0.9090287429945809
Epoch 6, Train Loss: 0.9558532426231786, Validation Loss: 0.8880487339837211
Epoch 7, Train Loss: 0.9366461258185538, Validation Loss: 0.8734409894262042
Epoch 8, Train Loss: 0.9222105804242586, Validation Loss: 0.8664211119924273
Epoch 9, Train Loss: 0.9132043468324762, Validation Loss: 0.8567688379968915
Epoch 10, Train Loss: 0.906146272232658, Validation Loss: 0.8531199097633362
Epoch 11, Train Loss: 0.8990879780367801, Validation Loss: 0.8465749110494342
Epoch 12, Train Loss: 0.8913789140550714, Validation Loss: 0.8413337809698922
Epoch 13, Train Loss: 0.8858821548913655, Validation Loss: 0.8371955156326294

(0.8428314547789725, 0.8134488633700779)

In [15]:
model2.evaluate(test_loader)

Accuracy: 0.6781313131313131


0.6781313131313131

The 68% OS accuracy looks much better! 

## Excercise.
Try different values of hyperparameters (64 triplets): num_layers = 1, 2, 3, 4; nhead = 1, 2, 4, 8; hidden_size = 16, 32, 64, 128. Compare performance (loss) after 10 epochs over the validation set. Pick the best configuration, and report the corresponding model's performance (loss and accuracy) over the test set after 30 training epochs